In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('공공보건의료기관현황.csv', index_col = 0, encoding='euc_kr')
print(data.columns)

Index(['병원 명', '설립형태', '근거 법령', '관계 행정기관', '관계 공공단체', '심평원 요양기관번호', '종별구분',
       '병상수', '소재지 우편번호', '주소', '홈페이지', '대표전화', 'FAX', '비고'],
      dtype='object')


In [2]:
addr = pd.DataFrame(data['주소'])
addr = addr['주소'].apply(lambda v: v.split()[:2])
addr = pd.DataFrame(addr.tolist(), columns=('시도', '군구'))
addr

,시도,군구
0,강원도,춘천시
1,강원도,삼척시
2,강원도,영월군
3,강원도,원주시
4,강원도,강릉시
...,...,...
222,충청북도,제천시
223,충청북도,충주시
224,충청북도,충주시
225,충청북도,영동군


In [3]:
print(addr['시도'].unique())

['강원도' '경기도' '경기' '경남' '창원시' '경상남도' '경상북도' '경산시' '경북' '인천광역시' '대구광역시'
 '전라남도' '대전광역시' '광주광역시' '제주특별자치도' '부산광역시' '전라북도' '충북' '서울특별시' '서울시'
 '부산특별시' '대전시' '충남' '전남' '충청남도' '울산광역시' '전북' '천안시' '충청북도']


In [4]:
# 창원시 주소 데이터 확인
addr[addr['시도'] == '창원시']

,시도,군구
27,창원시,의창구
31,창원시,마산합포구3.15대로


In [5]:
# 주소 데이터 변경(경상남도 창원시)
addr.iloc[27] = ['경상남도', '창원시']
addr.iloc[31] = ['경상남도', '창원시']
# 변경된 내용 확인
addr.iloc[27]

시도    경상남도
군구     창원시
Name: 27, dtype: object

In [6]:
# 경산시 변경
addr.iloc[47] = ['경상북도', '경산시']
# 천안시 변경
addr.iloc[209] = ['충청남도', '천안시']
addr.iloc[210] = ['충청남도', '천안시']

In [7]:
# 표준 행정구역 이름으로 수정 :경기 ->경기도,경남 ->경상남도,...

addr_aliases ={'경기':'경기도','경남':'경상남도','경북':'경상북도',
'충북':'충청북도','서울시':'서울특별시','부산특별시':'부산광역시',
'대전시':'대전광역시','충남':'충청남도','전남':'전라남도','전북':'전라북도'}

# dict.get(key[,default])
# - key에 해당하는 값이 없으면 default값을 리턴
addr['시도']=addr['시도'].apply(lambda v: addr_aliases.get(v,v))

In [8]:
print(addr['시도'].unique())

['강원도' '경기도' '경상남도' '경상북도' '인천광역시' '대구광역시' '전라남도' '대전광역시' '광주광역시'
 '제주특별자치도' '부산광역시' '전라북도' '충청북도' '서울특별시' '충청남도' '울산광역시']


In [9]:
# 아란 13길을 제주특별자치도, 제주시로 수정
addr.iloc[75] = ['제주특별자치도', '제주시']

In [11]:
# 행정구역별 공공보건 의료기관의 수 구하기
## '시도'와 '군구' 컬럼을 합친 새로운 컬럼추가
addr['시도군구'] = addr.apply(lambda r: r['시도'] +  ' ' + r['군구'], axis = 1)
addr.head()

,시도,군구,시도군구
0,강원도,춘천시,강원도 춘천시
1,강원도,삼척시,강원도 삼척시
2,강원도,영월군,강원도 영월군
3,강원도,원주시,강원도 원주시
4,강원도,강릉시,강원도 강릉시


In [12]:
# coundt 컬럼 추가
addr['count'] = 0
addr.head()

,시도,군구,시도군구,count
0,강원도,춘천시,강원도 춘천시,0
1,강원도,삼척시,강원도 삼척시,0
2,강원도,영월군,강원도 영월군,0
3,강원도,원주시,강원도 원주시,0
4,강원도,강릉시,강원도 강릉시,0


In [15]:
# 시도, 군구, 시도군구를 기준으로 그룹 생성
addr_group = pd.DataFrame(addr.groupby(['시도', '군구', '시도군구'],
                                       as_index = False).count())
addr_group

,시도,군구,시도군구,count
0,강원도,강릉시,강원도 강릉시,4
1,강원도,동해시,강원도 동해시,1
2,강원도,삼척시,강원도 삼척시,1
3,강원도,속초시,강원도 속초시,1
4,강원도,양구군,강원도 양구군,1
...,...,...,...,...
135,충청북도,영동군,충청북도 영동군,1
136,충청북도,제천시,충청북도 제천시,1
137,충청북도,청원군,충청북도 청원군,1
138,충청북도,청주시,충청북도 청주시,4


## 행정구역별 인구 데이터 준비

In [17]:
population = pd.read_excel('행정구역시군구별_성별인구수.xlsx')
population.head()

,행정구역(시군구)별(1),행정구역(시군구)별(2),총인구수 (명),남자인구수 (명),여자인구수 (명)
0,전국,소계,51847509,25862863,25984646
1,서울특별시,소계,9733509,4745088,4988421
2,서울특별시,종로구,151215,73688,77527
3,서울특별시,중구,126201,61946,64255
4,서울특별시,용산구,229385,110701,118684


In [19]:
# 컬럼의 이름 변경
population = population.rename(columns =
                               {'행정구역(시군구)별(1)': '시도',
                               '행정구역(시군구)별(2)': '군구'})

population.head()

,시도,군구,총인구수 (명),남자인구수 (명),여자인구수 (명)
0,전국,소계,51847509,25862863,25984646
1,서울특별시,소계,9733509,4745088,4988421
2,서울특별시,종로구,151215,73688,77527
3,서울특별시,중구,126201,61946,64255
4,서울특별시,용산구,229385,110701,118684


In [20]:
# 행정구역별 데이터에서 병합에 사용할 공통 컬럼 추가
# ['군구'] 컬럼에서 공백 제거
row_count = population.shape[0] #row의 개수
for element in range(0, row_count):
    population['군구'][element] = population['군구'][element].strip()

C:\Users\LJH\AppData\Local\Temp\ipykernel_1312\2484757658.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population['군구'][element] = population['군구'][element].strip()


In [22]:
# '시도군구' 컬럼 추가
population['시도군구']=population.apply(lambda r: r['시도'] + ' ' + r['군구'], axis = 1)
population.head()

,시도,군구,총인구수 (명),남자인구수 (명),여자인구수 (명),시도군구
0,전국,소계,51847509,25862863,25984646,전국 소계
1,서울특별시,소계,9733509,4745088,4988421,서울특별시 소계
2,서울특별시,종로구,151215,73688,77527,서울특별시 종로구
3,서울특별시,중구,126201,61946,64255,서울특별시 중구
4,서울특별시,용산구,229385,110701,118684,서울특별시 용산구


In [24]:
# ['군구] 컬럼에서 '소계' 항족 제거
population = population[population.군구 != '소계']

In [26]:
# ['시도군구'] 컬럼을 병합에 사용할 인덱스로 설정
population = population.set_index('시도군구')
population.head()

,시도,군구,총인구수 (명),남자인구수 (명),여자인구수 (명)
시도군구,,,,,
서울특별시 종로구,서울특별시,종로구,151215,73688,77527
서울특별시 중구,서울특별시,중구,126201,61946,64255
서울특별시 용산구,서울특별시,용산구,229385,110701,118684
서울특별시 성동구,서울특별시,성동구,300410,147020,153390
서울특별시 광진구,서울특별시,광진구,351263,170164,181099


### 데이터 병합
#### 내부조인
        둘 이상의 데이터프레임에서 조건에 맞는 행을 연결
#### 외부조인
        한쪽 데이터프레임에만 존재하는 데이터를 다른 데이터프레임에 결합

### merge 메서드
#### DataFrame.merge(right,	how='inner',	left_on=None,	right_on=None,	left_index=True,	right_index=True)

    데이터프레임 연결: 왼쪽 데이터프레임 기준
    • left_on,	right_on: 왼쪽, 오른쪽 데이터프레임에서 값이 일치해야 할 항목
    • left_index=True, right_index=True: 인덱스를 기준으로 병합       

In [29]:
# 의료기관 데이터와 행정구역 인구수 데이터 병합
addr_population_merge = pd.merge(addr_group, population, how = 'inner', left_index = True, right_index = True)
addr_population_merge

,시도_x,군구_x,시도군구,count,시도_y,군구_y,총인구수 (명),남자인구수 (명),여자인구수 (명)
